In [13]:
import os
import pandas as pd
import re
import pickle as pkl

#Preprocesamiento
import spacy
from spacy.lang.es.stop_words import STOP_WORDS #importar set de stopwords
from nltk.stem import SnowballStemmer #importar stemmer
nlp = spacy.load('es') #python -m spacy download es

#Bag-of-words
from sklearn.feature_extraction.text import CountVectorizer  

# Función de tokenización

In [14]:
def tokenizer(doc, sep=None, vocabulary = None, homol_dict=None, lemmatization=False, stemming = False):
    '''
    Por defecto divide la sentencia por el carácter espacio.
    Ej: 'Data Mining is the best course'->['Data',  'Mining', 'is', 'the', 'best', 'course']
    
    Input: 
    1. doc: str, documento.
    2. sep: str, carácter para dividir el documento en tokens, por defecto es el espacio.
    3. vocabuary: set, si un vocabulario es dado filtra las palabras que no estan presentes en el.
    4. homol_dict: dict, diccionario de homologaciones.
    5. lemmatization: bool, si es True lleva las palabras a su lema.
    6. stemming: bool, si es True lleva las palabas a su raíz.
    
    Output: 
    list, lista de tokens.
    '''
    doc = re.sub(r'\S+@\S+', '', doc) #elimina correos electrónicos  
    doc = re.sub(r'[\xa0]', '', doc) #elimina el patrón \xa0
    doc = re.sub(r'[^\w\s]','', doc) #elimina los símbolos de puntuación excepto underscore
    doc = re.sub(r'[_]', '', doc) #elimina underscore
    doc = re.sub(r'[a-zA-Z]+[0-9]+', '', doc) #elimina los tokens que contienen letras y números
    doc = re.sub(r'([ø ÿ þ])', ' ', doc) #reemplazas los símbolos contenidos por un espacio 
    doc = re.sub(r'[0-9]', '', doc) #elimina los tokens númericos
    tokens = doc.split(sep) #tokenización
    tokens = [word.lower() for word in tokens] #pasar todas las palabras a minúsculas
    
    
    if vocabulary is not None:
        #solo considera caracteres que tokens que estan en vocabulary
        tokens = [word for word in tokens if word in vocabulary]
    
    #if homol_dict is not None:
    
    if lemmatization==True:
        tokens = [nlp(word)[0].lemma_ for word in tokens]
        
    if stemming == True:
        stemmer = SnowballStemmer('spanish')
        tokens = [stemmer.stem(word) for word in tokens]
    
    return tokens

In [17]:
## Importar la base de datos
df = pd.read_csv('../data/robos_prose.csv', index_col = 'id_prose', usecols = ['id_prose', 'sin_fecha_siniestro', 'sin_relato'], sep=',')

# cambiar el tipo de dato a timestamp
df['sin_fecha_siniestro'] = pd.to_datetime(df['sin_fecha_siniestro'])

# ordenar relatos por fecha
df.sort_values('sin_fecha_siniestro', inplace=True)

# seleccionar relatos entre 2011-2016 y omitir registros con relatos nulos
df = df[(df['sin_fecha_siniestro']>=pd.Timestamp(2011,1,1)) & (df['sin_fecha_siniestro']<pd.Timestamp(2017,1,1)) & (df['sin_relato'].isnull()==False)]

# exportar dataframe
df.to_pickle('../data/robos_prose_v1.pkl')

#Importar la base de datos
df = pd.read_pickle('../data/robos_prose_v1.pkl')
df.head()

,sin_fecha_siniestro,sin_relato
id_prose,,
33474,2011-01-01,Por medio de la presente comunicamos a uds....
32730,2011-01-01,ESTABA ESTACIONADO EL LA CALLE ROTEMBURGO ENTR...
33469,2011-01-01,Estacionado al volver no encontre el vehiculo...
33466,2011-01-01,El abuelo de mi polola salio del domicilio y ...
32220,2011-01-01,ESTANDO ESTACIONADO VEHICULO EN AVDA PACIFICO ...


# Procesamiento

In [18]:
#Creamos objetos para llevar el corpus a bag-of-words
tf_vectorizer = CountVectorizer(analyzer='word', tokenizer= tokenizer)

#extraer vocabulario
tf_vectorizer.fit(df['sin_relato'])
vocabulary = tf_vectorizer.get_feature_names()
frequency = tf_vectorizer.transform(df['sin_relato']).toarray().sum(axis=0)
data = {'vocabulary':pd.Series(vocabulary), 'frequency': pd.Series(frequency)}
df_freq = pd.DataFrame(data)

In [859]:
# Extraer stopwords contextuales: palabras muy frecuentes que aportan poca información
stopwords_filter = ['veh', 'vh', 'culo', 'aut', 'camion', 'camión','rob', 'denun', 'dej', 
                    'daño', 'dano', 'daos', 'perc' ,'sinies' ,'llev' ,'volver' ,'sali' ,
                    'hrs' ,'dedu' ,'hecho' , 'habia', 'busc' ,'regre' ,'aseg' ,
                    'frent' ,'comuna' ,'direcc' ,'aprox' ,'circun' ,'lleg' , 'afuera' ,
                    'hora' ,'indica' ,'ubica' ,'minut' ,'conta', 'presen', 'senal', 
                    'señal', 'web', 'descrip', 'carabi', 'fech', 'avis', 'docu', 'ppu',
                    'dya', 'mario', 'medina', 'alcoholemiano'
                   ]
#ppu:patente, web: página web para denunciar

contextual_stopwords = set()

for word in stopwords_filter:
    words_to_filter = set(df_freq[df_freq['vocabulary'].str.contains(word)]['vocabulary'])
    contextual_stopwords = contextual_stopwords.union(words_to_filter)

not_stopwords = set(['autopista', 'autopistas', 'autoservicio', 'automático', 'algarrobo', 'petrobras'])
contextual_stopwords = contextual_stopwords-not_stopwords
contextual_stopwords = contextual_stopwords.union(set(['via', 'vía', 'uf', 'numero', 'número']))

# Nuevo conjunto de stopwords
stopwords_set = STOP_WORDS.union(contextual_stopwords)

# Extraer palabras que aparecen almenos 10 veces en el corpus y tienen más de tres caracteres
most_freq_words = set(df_freq[(df_freq['frequency']>=10) & (df_freq['vocabulary'].str.len()>3)]['vocabulary'])

# Vocabulario final corresponde most_freq_words menos los elementos en stopwords_set
vocabulary = most_freq_words-stopwords_set

In [864]:
with open('../data/vocabulary.pickle', 'wb') as f:
    pkl.dump(vocabulary, f, protocol = pkl.HIGHEST_PROTOCOL)

In [865]:
with open('../data/vocabulary.pickle', 'rb') as f:
    vocabulary = pkl.load(f)

In [866]:
vocabulary

{'mantiene',
 'easy',
 'daran',
 'piramide',
 'septiembre',
 'cuado',
 'devuelvo',
 'taller',
 'ocurrio',
 'noticia',
 'ventanilla',
 'efectuando',
 'terrano',
 'pare',
 'tirones',
 'jcgonzalez',
 'interseptado',
 'devuelve',
 'despertarme',
 'circulando',
 'costados',
 'diversas',
 'intimidar',
 'relatocallcenterexternoammg',
 'alfredo',
 'asomamos',
 'barra',
 'recuperación',
 'conserjeria',
 'chapa',
 'raspones',
 'alessandri',
 'aparec',
 'pistas',
 'diriji',
 'exito',
 'funeral',
 'iban',
 'fotografias',
 'acercaron',
 'focos',
 'alto',
 'comprando',
 'valdivia',
 'principal',
 'raquel',
 'mazda',
 'sala',
 'preparando',
 'escaparon',
 'bloquea',
 'residencial',
 'sufrio',
 'inferior',
 'llanta',
 'intimidada',
 'romper',
 'srta',
 'cosntancia',
 'vienen',
 'carroceria',
 'pdañadas',
 'patentes',
 'valparaiso',
 'retiro',
 'granaderos',
 'colorado',
 'corr',
 'xxiii',
 'estaciionado',
 'estacione',
 'ventas',
 'porteria',
 'levanta',
 'chiloe',
 'rodeo',
 'mencioanda',
 'paulina',

In [844]:
args = {"vocabulary": vocabulary, 'stemming': False}
#Creamos objetos para llevar el corpus a bag-of-words
tf_vectorizer = CountVectorizer(analyzer='word', tokenizer=lambda text: tokenizer(text, **args))

#extraer vocabulario
tf_vectorizer.fit(df['sin_relato'])
vocabulary = tf_vectorizer.get_feature_names()
frequency = tf_vectorizer.transform(df['sin_relato']).toarray().sum(axis=0)
data = {'vocabulary':pd.Series(vocabulary), 'frequency': pd.Series(frequency)}
df_freq = pd.DataFrame(data)

# To Do

Ver como capturar patrón horario, usando intervalos horarios a partir de números o palabras:
- tiempo = [ 'am', 'mañana', 'manana', 'pm', 'noche', 'madrugada', 'anoche']

Crear diccionario de homologaciones. Ejemplo:
- estacionar, bajar
- arma_fuego: [pistola, arma de fuego, ..]
- sujetos, tipos, individiuos, persona, tipo, personas bajar

Trabajar en la generación de frases, ejemplo:
- cajero_automático = [cajero automático]
- estacion_central = ['estación central', 'estacion central']
- estacion_de_servicio = ['estación de servicio', 'estacion de servicio', 'autoservicio', 'estacion de autoservicio ']
- estacion_del_metro = ['estación del metro']

Consideraciones
- Correción ortográfica: embeddings (60 ms) y permutaciones (230 ms).
- Lematización: spacy (5 ms), CREA (137 ns), se cae con faltas ortográficas
- Palabras cortadas: realizar búsqueda exhaustiva de carácteres que cortan palabras.